In [9]:
import os.path, csv, glob, time
import import_ipynb
from keras.models import Input
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.preprocessing import image
from Extractor import Extractor
from R_Extractor import R_Extractor
import tensorflow as tf
import numpy as np
import loupe_keras as lpk
from Model import MyNet
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.utils import to_categorical
from keras.applications.inception_v3 import InceptionV3, preprocess_input

def getFrames(name,game,action):
        root_path = 'SoccerNet/SoccerNet-code/data/data'          
        game_time = name[3]+'_'+name[6:11] 
        name= action+game_time
        specific_path =os.path.join(action,game,game_time)
        path = os.path.join(root_path, specific_path)         
        images = sorted(glob.glob(os.path.join(path, name+'*jpg')))         
        return images  
def rescaleList(input_list, size):        
        assert len(input_list) >= size
        skip = len(input_list) // size
        output = [input_list[i] for i in range(0, len(input_list), skip)]
        return output[:size]
def get_class_one_hot(class_str):
    if class_str == 'crd':
        label_encoded = 0
    elif class_str == 'scb':
        label_encoded = 1
    elif class_str == 'sbs':
        label_encoded = 2
    label_hot = to_categorical(label_encoded, 3) 
    assert len(label_hot) == 3
    return label_hot
    
sequence_length = 100 
emodel = 'I_Ex'
if emodel == 'I_Ex':
    exmodel=Extractor()
    feat_shape= (0,2048)
elif emodel == 'R_Ex':
    exmodel= R_Extractor()
    feat_shape= (0,2048)
trainer= MyNet(emodel)    



r = open("file_train.txt","r")
sampleList = []
labelList = []
numero_righe=0
for line in r:
    numero_righe= numero_righe+1
#pbar = tqdm(total = numero_righe)
pbar = tqdm(total = 4584 )

checkpointer = ModelCheckpoint(filepath= os.path.join('checkpoints',emodel+'-'
                                                      +'.{epoch:03d}-{val_loss:.3f}.hdf5'),
                               verbose=1,
                               save_best_only=True)
tb = TensorBoard(log_dir=os.path.join('logs', emodel))
early_stopper = EarlyStopping(patience=11)
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('logs', emodel + '-' + 'training-' + \
                                    str(timestamp) + '.log'))
r = open("file_train.txt","r")

y=0
for line in r:
    if y<3210 :
        y=y+1
        file_name , game_code = line.split(',')
        code_action = file_name[:3]
        game_code = game_code[:33]
        frames = getFrames(file_name,game_code,code_action)    
        frames = rescaleList(frames, sequence_length)    
        sequence =np.empty(feat_shape, dtype=object)
        for image in frames:  
            features= exmodel.extract(image) #restituisce un [(8,8,2048)]        
            features = np.expand_dims(features, axis=0)         
            sequence = np.concatenate( (sequence, features), axis=0) 
           
        sampleList.append(sequence)
        labelList.append(get_class_one_hot(code_action))
        pbar.update(1)

X=np.array(sampleList)
Y= np.array(labelList)
v = open("file_validation.txt","r")
y=0
sampleList = []
labelList = []
for line in v:
    if y<1374 :
        y=y+1
        file_name , game_code = line.split(',')
        code_action = file_name[:3]
        game_code = game_code[:33]
        frames = getFrames(file_name,game_code,code_action)    
        frames = rescaleList(frames, sequence_length)    
        sequence =np.empty(feat_shape, dtype=object)
        for image in frames:  
            features= exmodel.extract(image) #restituisce un [(8,8,2048)]             
            features = np.expand_dims(features, axis=0)         
            sequence = np.concatenate( (sequence, features), axis=0)            
        sampleList.append(sequence)
        labelList.append(get_class_one_hot(code_action))
        pbar.update(1)
pbar.close()        
X2=np.array(sampleList)
Y2= np.array(labelList) 
trainer.modello.fit(X,
                   Y,
                   validation_data=(X2,Y2),
                   epochs=100,
                   callbacks=[tb,early_stopper,csv_logger,checkpointer])





  0%|          | 0/4584 [00:00<?, ?it/s]


  0%|          | 1/4584 [00:33<42:08:14, 33.10s/it]


  0%|          | 2/4584 [00:45<34:11:25, 26.86s/it]


  0%|          | 3/4584 [00:57<28:38:11, 22.50s/it]


  0%|          | 4/4584 [01:10<24:48:27, 19.50s/it]


  0%|          | 5/4584 [01:22<22:04:53, 17.36s/it]


  0%|          | 6/4584 [01:35<20:14:17, 15.91s/it]


  0%|          | 7/4584 [01:47<18:54:53, 14.88s/it]


  0%|          | 8/4584 [02:00<17:58:33, 14.14s/it]


  0%|          | 9/4584 [02:12<17:15:03, 13.57s/it]


  0%|          | 10/4584 [02:24<16:43:41, 13.17s/it]


  0%|          | 11/4584 [02:36<16:24:26, 12.92s/it]


  0%|          | 12/4584 [02:49<16:08:40, 12.71s/it]


  0%|          | 13/4584 [03:01<15:57:55, 12.57s/it]


  0%|          | 14/4584 [03:13<15:50:01, 12.47s/it]


  0%|          | 15/4584 [03:25<15:46:43, 12.43s/it]


  0%|          | 16/4584 [03:38<15:39:13, 12.34s/it]


  0%|          | 17/4584 [03:50<15:42:10, 12.38s/it]


  0%|          | 18/4584 [04

  3%|▎         | 127/4584 [26:13<15:10:27, 12.26s/it]


  3%|▎         | 128/4584 [26:25<15:07:16, 12.22s/it]


  3%|▎         | 129/4584 [26:37<15:08:03, 12.23s/it]


  3%|▎         | 130/4584 [26:49<15:03:11, 12.17s/it]


  3%|▎         | 131/4584 [27:01<15:02:57, 12.17s/it]


  3%|▎         | 132/4584 [27:13<14:59:21, 12.12s/it]


  3%|▎         | 133/4584 [27:26<14:58:45, 12.12s/it]


  3%|▎         | 134/4584 [27:38<14:59:07, 12.12s/it]


  3%|▎         | 135/4584 [27:50<15:01:31, 12.16s/it]


  3%|▎         | 136/4584 [28:02<15:02:55, 12.18s/it]


  3%|▎         | 137/4584 [28:14<15:01:10, 12.16s/it]


  3%|▎         | 138/4584 [28:26<15:01:21, 12.16s/it]


  3%|▎         | 139/4584 [28:38<14:57:36, 12.12s/it]


  3%|▎         | 140/4584 [28:51<14:55:47, 12.09s/it]


  3%|▎         | 141/4584 [29:03<14:54:47, 12.08s/it]


  3%|▎         | 142/4584 [29:15<14:54:54, 12.09s/it]


  3%|▎         | 143/4584 [29:27<14:54:05, 12.08s/it]


  3%|▎         | 144/4584 [29:39<14:55:11, 12.10

  6%|▌         | 253/4584 [51:36<14:35:17, 12.13s/it]


  6%|▌         | 254/4584 [51:48<14:34:10, 12.11s/it]


  6%|▌         | 255/4584 [52:01<14:33:49, 12.11s/it]


  6%|▌         | 256/4584 [52:13<14:33:22, 12.11s/it]


  6%|▌         | 257/4584 [52:25<14:31:51, 12.09s/it]


  6%|▌         | 258/4584 [52:36<14:23:50, 11.98s/it]


  6%|▌         | 259/4584 [52:49<14:27:51, 12.04s/it]


  6%|▌         | 260/4584 [53:01<14:27:28, 12.04s/it]


  6%|▌         | 261/4584 [53:13<14:34:09, 12.13s/it]


  6%|▌         | 262/4584 [53:25<14:31:27, 12.10s/it]


  6%|▌         | 263/4584 [53:37<14:28:12, 12.06s/it]


  6%|▌         | 264/4584 [53:49<14:31:37, 12.11s/it]


  6%|▌         | 265/4584 [54:01<14:30:47, 12.10s/it]


  6%|▌         | 266/4584 [54:13<14:29:18, 12.08s/it]


  6%|▌         | 267/4584 [54:26<14:30:49, 12.10s/it]


  6%|▌         | 268/4584 [54:38<14:31:01, 12.11s/it]


  6%|▌         | 269/4584 [54:50<14:31:41, 12.12s/it]


  6%|▌         | 270/4584 [55:02<14:25:56, 12.04

  8%|▊         | 376/4584 [1:16:23<14:12:52, 12.16s/it]


  8%|▊         | 377/4584 [1:16:35<14:11:30, 12.14s/it]


  8%|▊         | 378/4584 [1:16:47<14:08:43, 12.11s/it]


  8%|▊         | 379/4584 [1:16:59<14:08:58, 12.11s/it]


  8%|▊         | 380/4584 [1:17:12<14:06:37, 12.08s/it]


  8%|▊         | 381/4584 [1:17:24<14:04:47, 12.06s/it]


  8%|▊         | 382/4584 [1:17:35<14:02:40, 12.03s/it]


  8%|▊         | 383/4584 [1:17:48<14:05:33, 12.08s/it]


  8%|▊         | 384/4584 [1:18:00<14:06:43, 12.10s/it]


  8%|▊         | 385/4584 [1:18:12<14:08:34, 12.13s/it]


  8%|▊         | 386/4584 [1:18:24<14:06:55, 12.10s/it]


  8%|▊         | 387/4584 [1:18:36<14:04:08, 12.07s/it]


  8%|▊         | 388/4584 [1:18:48<14:03:21, 12.06s/it]


  8%|▊         | 389/4584 [1:19:00<14:02:39, 12.05s/it]


  9%|▊         | 390/4584 [1:19:12<14:03:13, 12.06s/it]


  9%|▊         | 391/4584 [1:19:24<14:02:23, 12.05s/it]


  9%|▊         | 392/4584 [1:19:36<14:01:46, 12.05s/it]


  9%|▊        

 11%|█         | 498/4584 [1:40:55<13:39:24, 12.03s/it]


 11%|█         | 499/4584 [1:41:07<13:37:46, 12.01s/it]


 11%|█         | 500/4584 [1:41:19<13:34:41, 11.97s/it]


 11%|█         | 501/4584 [1:41:30<13:31:04, 11.92s/it]


 11%|█         | 502/4584 [1:41:42<13:34:50, 11.98s/it]


 11%|█         | 503/4584 [1:41:54<13:35:13, 11.99s/it]


 11%|█         | 504/4584 [1:42:06<13:34:41, 11.98s/it]


 11%|█         | 505/4584 [1:42:19<13:36:53, 12.02s/it]


 11%|█         | 506/4584 [1:42:30<13:33:59, 11.98s/it]


 11%|█         | 507/4584 [1:42:42<13:32:50, 11.96s/it]


 11%|█         | 508/4584 [1:42:54<13:34:43, 11.99s/it]


 11%|█         | 509/4584 [1:43:06<13:32:03, 11.96s/it]


 11%|█         | 510/4584 [1:43:18<13:29:23, 11.92s/it]


 11%|█         | 511/4584 [1:43:30<13:30:40, 11.94s/it]


 11%|█         | 512/4584 [1:43:42<13:33:07, 11.98s/it]


 11%|█         | 513/4584 [1:43:54<13:33:57, 12.00s/it]


 11%|█         | 514/4584 [1:44:06<13:35:40, 12.02s/it]


 11%|█        

...

 90%|█████████ | 4132/4584 [11:46:08<1:01:38,  8.18s/it]


 90%|█████████ | 4133/4584 [11:46:16<1:01:20,  8.16s/it]


 90%|█████████ | 4134/4584 [11:46:24<1:01:18,  8.18s/it]


 90%|█████████ | 4135/4584 [11:46:32<1:00:52,  8.13s/it]


 90%|█████████ | 4136/4584 [11:46:40<1:00:45,  8.14s/it]


 90%|█████████ | 4137/4584 [11:46:48<1:00:46,  8.16s/it]


 90%|█████████ | 4138/4584 [11:46:57<1:00:35,  8.15s/it]


 90%|█████████ | 4139/4584 [11:47:05<1:00:25,  8.15s/it]


 90%|█████████ | 4140/4584 [11:47:13<1:00:33,  8.18s/it]


 90%|█████████ | 4141/4584 [11:47:21<1:00:20,  8.17s/it]


 90%|█████████ | 4142/4584 [11:47:29<1:00:09,  8.17s/it]


 90%|█████████ | 4143/4584 [11:47:38<1:00:23,  8.22s/it]


 90%|█████████ | 4144/4584 [11:47:46<1:00:09,  8.20s/it]


 90%|█████████ | 4145/4584 [11:47:54<1:00:02,  8.21s/it]


 90%|█████████ | 4146/4584 [11:48:02<59:41,  8.18s/it]  


 90%|█████████ | 4147/4584 [11:48:10<59:26,  8.16s/it]


 90%|█████████ | 4148/4584 [11:48:18<59:25,  8.18s/it]


 

 93%|█████████▎| 4255/4584 [12:02:52<44:49,  8.17s/it]


 93%|█████████▎| 4256/4584 [12:03:00<44:32,  8.15s/it]


 93%|█████████▎| 4257/4584 [12:03:08<44:23,  8.14s/it]


 93%|█████████▎| 4258/4584 [12:03:16<44:14,  8.14s/it]


 93%|█████████▎| 4259/4584 [12:03:25<44:05,  8.14s/it]


 93%|█████████▎| 4260/4584 [12:03:33<43:56,  8.14s/it]


 93%|█████████▎| 4261/4584 [12:03:41<43:49,  8.14s/it]


 93%|█████████▎| 4262/4584 [12:03:49<43:47,  8.16s/it]


 93%|█████████▎| 4263/4584 [12:03:57<43:39,  8.16s/it]


 93%|█████████▎| 4264/4584 [12:04:05<43:40,  8.19s/it]


 93%|█████████▎| 4265/4584 [12:04:14<43:29,  8.18s/it]


 93%|█████████▎| 4266/4584 [12:04:22<43:27,  8.20s/it]


 93%|█████████▎| 4267/4584 [12:04:30<43:18,  8.20s/it]


 93%|█████████▎| 4268/4584 [12:04:38<43:09,  8.20s/it]


 93%|█████████▎| 4269/4584 [12:04:46<42:59,  8.19s/it]


 93%|█████████▎| 4270/4584 [12:04:55<42:50,  8.19s/it]


 93%|█████████▎| 4271/4584 [12:05:03<42:35,  8.16s/it]


 93%|█████████▎| 4272/4584 [12:

 96%|█████████▌| 4379/4584 [12:19:45<28:04,  8.22s/it]


 96%|█████████▌| 4380/4584 [12:19:54<27:57,  8.22s/it]


 96%|█████████▌| 4381/4584 [12:20:02<27:48,  8.22s/it]


 96%|█████████▌| 4382/4584 [12:20:10<27:41,  8.22s/it]


 96%|█████████▌| 4383/4584 [12:20:18<27:35,  8.24s/it]


 96%|█████████▌| 4384/4584 [12:20:26<27:19,  8.20s/it]


 96%|█████████▌| 4385/4584 [12:20:35<27:12,  8.20s/it]


 96%|█████████▌| 4386/4584 [12:20:43<27:02,  8.19s/it]


 96%|█████████▌| 4387/4584 [12:20:51<26:56,  8.21s/it]


 96%|█████████▌| 4388/4584 [12:20:59<26:44,  8.19s/it]


 96%|█████████▌| 4389/4584 [12:21:07<26:44,  8.23s/it]


 96%|█████████▌| 4390/4584 [12:21:16<26:32,  8.21s/it]


 96%|█████████▌| 4391/4584 [12:21:24<26:23,  8.20s/it]


 96%|█████████▌| 4392/4584 [12:21:32<26:13,  8.20s/it]


 96%|█████████▌| 4393/4584 [12:21:40<26:06,  8.20s/it]


 96%|█████████▌| 4394/4584 [12:21:48<25:56,  8.19s/it]


 96%|█████████▌| 4395/4584 [12:21:57<25:51,  8.21s/it]


 96%|█████████▌| 4396/4584 [12:

 98%|█████████▊| 4503/4584 [12:36:39<11:00,  8.16s/it]


 98%|█████████▊| 4504/4584 [12:36:47<10:51,  8.14s/it]


 98%|█████████▊| 4505/4584 [12:36:55<10:44,  8.16s/it]


 98%|█████████▊| 4506/4584 [12:37:03<10:35,  8.14s/it]


 98%|█████████▊| 4507/4584 [12:37:11<10:25,  8.13s/it]


 98%|█████████▊| 4508/4584 [12:37:19<10:16,  8.12s/it]


 98%|█████████▊| 4509/4584 [12:37:28<10:10,  8.14s/it]


 98%|█████████▊| 4510/4584 [12:37:36<10:03,  8.16s/it]


 98%|█████████▊| 4511/4584 [12:37:44<09:54,  8.14s/it]


 98%|█████████▊| 4512/4584 [12:37:52<09:45,  8.13s/it]


 98%|█████████▊| 4513/4584 [12:38:00<09:37,  8.14s/it]


 98%|█████████▊| 4514/4584 [12:38:08<09:28,  8.13s/it]


 98%|█████████▊| 4515/4584 [12:38:16<09:20,  8.12s/it]


 99%|█████████▊| 4516/4584 [12:38:25<09:12,  8.13s/it]


 99%|█████████▊| 4517/4584 [12:38:33<09:04,  8.13s/it]


 99%|█████████▊| 4518/4584 [12:38:41<08:56,  8.14s/it]


 99%|█████████▊| 4519/4584 [12:38:49<08:48,  8.13s/it]


 99%|█████████▊| 4520/4584 [12:

Train on 3210 samples, validate on 1374 samples
Epoch 1/100
3210/3210 [==============================] - 64s 20ms/step - loss: 0.5157 - acc: 0.7494 - val_loss: 0.4326 - val_acc: 0.8062

Epoch 00001: val_loss improved from inf to 0.43259, saving model to checkpoints/I_Ex-.001-0.433.hdf5
Epoch 2/100
3210/3210 [==============================] - 64s 20ms/step - loss: 0.3552 - acc: 0.8537 - val_loss: 0.3409 - val_acc: 0.8549

Epoch 00002: val_loss improved from 0.43259 to 0.34093, saving model to checkpoints/I_Ex-.002-0.341.hdf5
Epoch 3/100
3210/3210 [==============================] - 56s 17ms/step - loss: 0.2773 - acc: 0.8908 - val_loss: 0.3021 - val_acc: 0.8763

Epoch 00003: val_loss improved from 0.34093 to 0.30213, saving model to checkpoints/I_Ex-.003-0.302.hdf5
Epoch 4/100
3210/3210 [==============================] - 57s 18ms/step - loss: 0.2332 - acc: 0.9120 - val_loss: 0.2839 - val_acc: 0.8845

Epoch 00004: val_loss improved from 0.30213 to 0.28389, saving model to checkpoints/I_Ex-

In [3]:
history_dict = history.history

loss_values = history_dict['acc']
val_loss_values = history_dict['val_acc']
epochs = range(1, len(history_dict['acc'])+1) 
plt.plot(epochs, loss_values, 'bo', label= 'Training accuracy')
plt.plot(epochs, val_loss_values, 'b', label='Validation accuracy')
plt.title('training and Validation Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()


NameError: name 'history' is not defined